In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
dataframe = pd.read_csv("https://www.dropbox.com/s/dzs9sqd783zfooh/iris.csv?dl=1")
dataframe.head()
print(dataframe)

train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')


def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('species')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

  
feature_columns = []

for header in ['sl', 'sw', 'pl', 'pw']:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)


batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)


loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

      sl   sw   pl   pw  species
0    6.7  3.3  5.7  2.5        2
1    7.2  3.6  6.1  2.5        2
2    6.2  2.8  4.8  1.8        2
3    7.7  3.0  6.1  2.3        2
4    5.8  2.7  5.1  1.9        2
..   ...  ...  ...  ...      ...
134  5.4  3.9  1.7  0.4        0
135  4.9  3.1  1.5  0.1        0
136  4.8  3.0  1.4  0.3        0
137  6.9  3.1  4.9  1.5        1
138  5.0  3.6  1.4  0.2        0

[139 rows x 5 columns]
88 train examples
23 validation examples
28 test examples

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/5
Instructions for upd